In [1]:
import numpy as np
import random
import math
from operator import mul,neg

from EstrategiaParticionado import ValidacionCruzada
from datos import Datos
from ClasificadorMulticlase import ClasificadorMulticlase
from sklearn.multiclass import OneVsRestClassifier
from Clasificador import Clasificador

from scipy.special import expit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn import datasets
from numpy.linalg import norm
from itertools import zip_longest




In [2]:
def producto_escalar(w,x):
    return sum(map(mul,w,x))


In [3]:
def sigmoidal (a):
    if a >= 700:
        return 1
    elif a <= -700:
        return 0 
    else:
        return 1 / (1 + math.exp(-a))

In [4]:
def crear_vector_w(d):
    return np.array([ random.uniform(-1,1) for i in range(d) ])


In [5]:
crear_vector_w(6)

array([ 0.14198184,  0.07547804, -0.79222264, -0.38273837, -0.60967963,
        0.60718926])

In [6]:
def crear_vector_datos(dato):
    return np.insert(dato,0,1)

In [7]:
crear_vector_datos(np.array([0,8,3,2]))

array([1, 0, 8, 3, 2])

In [8]:
vector = np.array([1,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189])
print(producto_escalar(crear_vector_w(len(vector)),vector))


1119.74530948


In [17]:
class ClasificadorRegresionLogistica(Clasificador):
    w = np.array([])
    n = 0.0 #cte aprendizaje
    num_epocas = 0

    def __init__(self,num_epocas, n):
        super(ClasificadorRegresionLogistica, self).__init__()
        self.num_epocas = num_epocas
        self.n = n
        
    def __generar_vector_nuevo(self,dato,sigmoidal,clase):
        factor = self.n * (sigmoidal - clase)
        return self.w - np.array(list(map(lambda x : x * factor, dato)))
    
    def entrenamiento(self, datosTrain, atributosDiscretos, diccionario):
        self.w = crear_vector_w(len(atributosDiscretos))
        for epoca in range(self.num_epocas):
            for dato in datosTrain:
                vector_datos = crear_vector_datos(dato[:-1])
                pe = producto_escalar( self.w,vector_datos)
                sigmoidal_res = sigmoidal(pe)
                self.w = self.__generar_vector_nuevo(vector_datos,
                                                     sigmoidal_res,
                                                     dato[-1])
        return self.w
    def __calcular_confianza(self,vector_dato_test):
        return producto_escalar(vector_dato_test,self.w) / norm(self.w)
    
    def __verosimilitud(self,vector_dato_test):
        return sigmoidal(producto_escalar(self.w,vector_dato_test))
    
    def __clasifica_dato(self,dato_test):
        return int(self.__verosimilitud(dato_test) > 0.5)
    
    def clasifica (self, datosTest, atributosDiscretos, diccionario):
        vectores_datos_test = map(crear_vector_datos,datosTest)
        return list(map(self.__clasifica_dato, vectores_datos_test))
    
    def __confianza(self, datosTest):
        vectores_datos_test = map(crear_vector_datos,datosTest)
        return [ self.__calcular_confianza(vector_dato_test)
                   for vector_dato_test in vectores_datos_test]
    
    def score (self, datosTest, atributosDiscretos, diccionario):
        scores = np.zeros((len(datosTest),2))
        predicciones = self.clasifica(datosTest,atributosDiscretos,diccionario)
        confianzas = self.__confianza(datosTest)
        for (pred,confianza,i) in zip_longest(predicciones,confianzas,range(datosTest.shape[0])):
            if (pred):
                scores[i,1] = confianza
                scores[i,0] = 1 - confianza
            else: 
                scores[i,1] = 1 - confianza 
                scores[i,0] = confianza           
        return scores


In [15]:
wdbc = Datos('../ficheros/ConjuntosDatos/wdbc.data',True)
example3 = Datos('../ficheros/ConjuntosDatos/example3.data',True)
example4 = Datos('../ficheros/ConjuntosDatos/example4.data',True)
wine = Datos('../ficheros/ConjuntosDatos/wine_proc.data',True)
digits = Datos('../ficheros/ConjuntosDatos/digits.data',True)
iris = datasets.load_iris()

In [11]:
estrategia = ValidacionCruzada(10)
sklearn_cv = KFold(n_splits=10,shuffle=True)
clasificador_sklearn = LogisticRegression() 


In [17]:
#clasificador = ClasificadorRegresionLogistica(10,1)

scores = cross_val_score(clasificador_sklearn, wdbc.datos[:,:-1],wdbc.datos[:,-1],cv = sklearn_cv)
print(np.mean(scores))

0.954260651629


In [18]:
X_train,X_test,y_train,y_test = train_test_split(wdbc.datos[:,:-1],
                                                 wdbc.datos[:,-1], 
                                                 test_size = 0.2)
clasificador_sklearn.fit(X_train,y_train)
print(clasificador_sklearn.score(X_test,y_test))

0.947368421053


In [19]:
print(len(wdbc.datos[0]))

31


In [20]:
X_train,X_test,y_train,y_test = train_test_split(example3.datos[:,:-1],
                                                 example3.datos[:,-1], 
                                                 test_size = 0.2)
clasificador_sklearn.fit(X_train,y_train)
print(clasificador_sklearn.score(X_test,y_test))

0.3125


In [27]:
clasificador = ClasificadorRegresionLogistica(1000,0.001)

In [28]:
errores = clasificador.validacion(estrategia,wdbc,clasificador)
print(1 - np.mean(errores))

0.89206043956


In [29]:
errores = clasificador.validacion(estrategia,example3,clasificador)

print(1 - np.mean(errores))

0.4575


In [30]:
errores = clasificador.validacion(estrategia,example4,clasificador)

print(1 - np.mean(errores))

1.0


In [20]:
clasificador = ClasificadorMulticlase(ClasificadorRegresionLogistica(1000,0.001))

In [19]:
errores = clasificador.validacion(estrategia,wine,clasificador)
print(1 - np.mean(errores))

0.269176470588


In [ ]:
errores = clasificador.validacion(estrategia,digits,clasificador)
print(1 - np.mean(errores))

In [45]:
clasificador_sklearn = OneVsRestClassifier(LogisticRegression())
X_train,X_test,y_train,y_test = train_test_split(digits.datos[:,:-1],
                                                 digits.datos[:,-1], 
                                                 test_size = 0.2)
clasificador_sklearn.fit(X_train,y_train)
print(clasificador_sklearn.score(X_test,y_test))

0.893617021277


In [42]:
clasificador_sklearn = OneVsRestClassifier(LogisticRegression())
X_train,X_test,y_train,y_test = train_test_split(wine.datos[:,:-1],
                                                 wine.datos[:,-1], 
                                                 test_size = 0.2)
clasificador_sklearn.fit(X_train,y_train)
print(clasificador_sklearn.score(X_test,y_test))

0.944444444444
